# GPT API to generate meme description

In [ ]:
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.7 MB/s eta 0:00:00


In [ ]:
# Download the EXIST 2024 Memes Dataset
# Please manually download the dataset from the official source and place it in the project directory
# Expected file: EXIST_2024_Memes_Dataset.zip

Downloading...
From (original): https://drive.google.com/uc?id=1ewvMPp9nsaTZ6tKcE-quLg5dnNMrbLF6
From (redirected): https://drive.google.com/uc?id=1ewvMPp9nsaTZ6tKcE-quLg5dnNMrbLF6&confirm=t&uuid=79402316-ebf4-4017-bfdd-d6f3e0c3950b
To: /content/EXIST 2024 Memes Dataset.zip
100% 711M/711M [00:28<00:00, 24.8MB/s]


In [ ]:
!unzip EXIST_2024_Memes_Dataset.zip -d ./data/

流式输出内容被截断，只能显示最后 5000 行内容。
  inflating: /content/EXIST 2024 Memes Dataset/training/memes/110601.jpeg  
  inflating: /content/__MACOSX/EXIST 2024 Memes Dataset/training/memes/._110601.jpeg  
  inflating: /content/EXIST 2024 Memes Dataset/training/memes/210654.jpeg  
  inflating: /content/__MACOSX/EXIST 2024 Memes Dataset/training/memes/._210654.jpeg  
  inflating: /content/EXIST 2024 Memes Dataset/training/memes/211946.jpeg  
  inflating: /content/__MACOSX/EXIST 2024 Memes Dataset/training/memes/._211946.jpeg  
  inflating: /content/EXIST 2024 Memes Dataset/training/memes/111440.jpeg  
  inflating: /content/__MACOSX/EXIST 2024 Memes Dataset/training/memes/._111440.jpeg  
  inflating: /content/EXIST 2024 Memes Dataset/training/memes/211415.jpeg  
  inflating: /content/__MACOSX/EXIST 2024 Memes Dataset/training/memes/._211415.jpeg  
  inflating: /content/EXIST 2024 Memes Dataset/training/memes/111010.jpeg  
  inflating: /content/__MACOSX/EXIST 2024 Memes Dataset/training/memes/._111010.jp

In [ ]:
# split the Spanish memes and English memes
import os
import shutil

# create two directories to store the memes
os.makedirs('./data/Spanish_memes', exist_ok=True)
os.makedirs('./data/English_memes', exist_ok=True)

memes_path = './data/EXIST 2024 Memes Dataset/training/memes'

for filename in os.listdir(memes_path):

    if filename.startswith('1'):

        shutil.copy(os.path.join(memes_path, filename), os.path.join('./data/Spanish_memes', filename))
    elif filename.startswith('2'):

        shutil.copy(os.path.join(memes_path, filename), os.path.join('./data/English_memes', filename))

In [ ]:
# create two directories to store the memes
os.makedirs('./data/Spanish_memes_test', exist_ok=True)

memes_path = './data/EXIST 2024 Memes Dataset/test/memes'

for filename in os.listdir(memes_path):

    if filename.startswith('3'):

        shutil.copy(os.path.join(memes_path, filename), os.path.join('./data/Spanish_memes_test', filename))

In [ ]:
import openai

In [ ]:
# Replace with your actual OpenAI API key
API_key = 'YOUR_OPENAI_API_KEY_HERE'

In [ ]:
openai.api_key = API_key

In [ ]:
import base64
import requests

# encode the image from the local file
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def analyze_image_for_sexist_content(image_path, API_key):
    base64_image = encode_image(image_path)
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_key}"
    }
    if 'English' in image_path:
      payload = {
          "model": "gpt-4-turbo",
          "temperature": 0.75,
          "seed": 1234,
          "messages": [
                  {"role": "system",
                  "content": "You are an expert in identifying the presence of sexism within memes. Your task is to assess whether the content of a meme portrays or criticizes sexism through marginalization, prejudice against women based on gender stereotypes, beliefs in male superiority, or misogynistic views."},

                  {"role": "user",
                  "content": [
                      {
                          "type": "text",
                          "text": "Briefly describe and analyze the content of the meme in one sentence, then label it as 'YES' if the meme embodies or criticizes sexism, or 'NO' if it does not. Please use the following format: 1. Description: [Your description]. 2. Label: [YES/NO]."
                      },
                      {
                          "type": "image_url",
                          "image_url":
                              "url": f"data:image/jpeg;base64,{base64_image}",
                              "detail": "low"
                          }
                      }
                  ]
              }
          ],
      }
      response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    elif 'Spanish' in image_path:
      payload = {
        "model": "gpt-4-turbo",
        "temperature": 0.75,
        "seed": 4321,
        "messages": [
                {"role": "system",
                "content": "Eres un experto en identificar la presencia de sexismo en los memes. Tu tarea es evaluar si el contenido de un meme retrata o critica el sexismo a través de la marginación, prejuicios contra las mujeres basados en estereotipos de género, creencias en la superioridad masculina o vistas misóginas."},

                {"role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Describa y analice brevemente el contenido del meme en una frase, luego etiquételo como 'YES' si el meme encarna o critica el sexismo, o 'NO' si no lo hace. Por favor, utilice el siguiente formato: 1. Description: [Tu descripción]. 2. Label: [YES/NO]."
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{base64_image}",
                            "detail": "low"  # use low resolution to process the image
                        }
                    }
                ]
            }
        ],
    }
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    return response.json()['choices'][0]['message']['content']


In [ ]:
example_path = './data/Spanish_memes_test/310531.jpeg'
example_result = analyze_image_for_sexist_content(example_path, API_key)
example_result

'1. Description: El meme muestra a un personaje con gafas de sol negando la existencia del machismo en un cuadro y en el siguiente admite que las acusaciones de machismo son un "confesión" de la lista de Vox por violencia machista.\n2. Label: YES'

In [ ]:
example_path = './data/Spanish_memes_test/310532.jpeg'
example_result = analyze_image_for_sexist_content(example_path, API_key)
example_result

'1. Description: El meme muestra a una mujer gritando con expresión enfática, el texto discute sobre la minimización de las agresiones machistas en los centros sociales y cómo esto debilita al movimiento feminista, con una respuesta enfática de "ZAS! EN TODA LA BOCA".\n2. Label: YES \n\nEl meme critica la actitud de minimizar las agresiones machistas, un problema real dentro del feminismo y la lucha por la igualdad de género, mostrando la inaceptabilidad de tales actitudes.'

In [ ]:
example_path = './data/Spanish_memes_test/310533.jpeg'
example_result = analyze_image_for_sexist_content(example_path, API_key)
example_result

'1. Description: El meme muestra una imagen de un pollito amarillo con texto que promueve la paz, la no violencia, la no discriminación y el rechazo a los discursos de odio.\n2. Label: NO.'

In [ ]:
example_path = './data/Spanish_memes_test/310535.jpeg'
example_result = analyze_image_for_sexist_content(example_path, API_key)
example_result

'1. Description: El meme muestra una imagen de Vegeta, un personaje del anime Dragon Ball, con un texto que se burla de una supuesta acción del "Ministerio de las Mujeres" para cancelar Dragon Ball por "violencia simbólica". El texto arriba del meme sugiere que Cartoon Network carece de alma por permitir la censura.\n2. Label: NO.'

In [ ]:
example_path = './data/Spanish_memes_test/310539.jpeg'
example_result = analyze_image_for_sexist_content(example_path, API_key)
example_result

'1. Description: El meme muestra una imagen de una mujer sonriendo con un texto superpuesto que dice "VANESSA" y debajo "ZORRA YO JIJI NO CREO".\n2. Label: YES\n\nEste meme parece utilizar un estereotipo negativo al referirse a una mujer como "zorra", lo cual puede ser interpretado como misógino y sexista al perpetuar un juicio devalor basado en el género.'

In [ ]:
example_path = './data/Spanish_memes_test/310540.jpeg'
example_result = analyze_image_for_sexist_content(example_path, API_key)
example_result

'1. Description: El meme muestra dos imágenes de una muñeca Barbie con diferentes expresiones faciales. En la primera, la muñeca tiene una expresión seria mientras aplica lápiz labial, con texto que dice "Cuando me dicen. // pta, pndja, zorra.,". En la segunda, la muñeca tiene una expresión de sorpresa, con el texto "Cuando me dicen SEÑORA".\n2. Label: YES\n\nEste meme destaca las reacciones diferenciadas a distintos tipos de tratamiento verbal hacia las mujeres, enfatizando una respuesta negativa a insultos misóginos y una reacción de sorpresa ante un término respetuoso, lo que refleja crítica hacia el sexismo y la forma en que las mujeres son tratadas o etiquetadas en la sociedad.'

In [ ]:
parts = example_result.split('\n')
parts

['1. Description: El meme muestra dos imágenes de una muñeca Barbie con diferentes expresiones faciales. En la primera, la muñeca tiene una expresión seria mientras aplica lápiz labial, con texto que dice "Cuando me dicen. // pta, pndja, zorra.,". En la segunda, la muñeca tiene una expresión de sorpresa, con el texto "Cuando me dicen SEÑORA".',
 '2. Label: YES',
 '',
 'Este meme destaca las reacciones diferenciadas a distintos tipos de tratamiento verbal hacia las mujeres, enfatizando una respuesta negativa a insultos misóginos y una reacción de sorpresa ante un término respetuoso, lo que refleja crítica hacia el sexismo y la forma en que las mujeres son tratadas o etiquetadas en la sociedad.']

In [ ]:
len(parts)

2

In [ ]:
import os
import pandas as pd

def create_df(path):
  images = [os.path.join(path, f) for f in os.listdir(path)]
  print(f"Total images: {len(images)}")

  sorted_images = sorted(images, key=lambda x: int(os.path.basename(x).split('.')[0]))

  answers = dict()
  results = []

  for image_path in sorted_images:
      try:
          response = analyze_image_for_sexist_content(image_path, API_key)
          print(os.path.basename(image_path), response)
          answers[os.path.basename(image_path)] = response
          if response:
              segments = response.split('\n')
              if len(segments) >= 2:
                  description = segments[0].replace('1. Description: ', '').strip()
                  label_info = segments[1].split(': ')
                  label = label_info[1].strip('.') if len(label_info) == 2 else 'Format error'
              else:
                  description = 'Response format error'
                  label = 'Unknown'
          else:
              description = 'No content returned'
              label = 'Unknown'
      except Exception as e:
          print(f"Error processing {image_path}: {str(e)}")
          description = 'Error in processing'
          label = 'Unknown'
      meme_id = os.path.basename(image_path)

      results.append({'id_EXIST': meme_id, 'description': description, 'reference': label})

  df = pd.DataFrame(results)
  return df, answers

In [ ]:
# process English meme and Spanish meme seperately
english_path = "./data/English_memes/"
english_memes_df, english_answers = create_df(english_path)

Total images: 2010
210001.jpeg 1. Description: The meme shows a man appearing distressed with a caption that expresses his disbelief that his partner cheated on him with another man, despite the fact that she previously cheated on her former boyfriend with him, ending with "SHAME ON ME!"
2. Label: YES

The meme can be seen as embodying sexism by promoting a stereotype that focuses on the untrustworthiness and moral failings of women in relationships. It suggests a pattern of behavior based on gender, which can perpetuate negative perceptions and mistrust towards women.
210002.jpeg 1. Description: The meme shows two basketball players looking skeptical or disapproving, captioned with a statement about disregarding the number of sexual partners a woman has had.
2. Label: YES
210003.jpeg 1. Description: The meme features a character from Game of Thrones with a caption that jokes about the importance of male genitalia, using the wordplay on "cocks" to mean both roosters and a colloquial te

In [ ]:
spanish_test = "./data/Spanish_memes_test/"
spanish_memes_df_test, spanish_answers_test = create_df(spanish_test)

Total images: 540
310001.jpeg 1. Description: El meme muestra a una mujer y un hombre con texto que dice "Soy como la madre de mi hermano". 
2. Label: NO.
310002.jpeg 1. Description: El video muestra a una mujer en Irán siendo reprendida por no llevar velo, con texto que indica la represión de mujeres y niñas que desafían la obligación de llevarlo.
2. Label: YES
310003.jpeg 1. Description: El meme muestra una imagen de una escena de "El Señor de los Anillos" donde Gandalf inspecciona un objeto brillante, con el texto "YA VIENEN." La siguiente imagen es una fotografía de una protesta de mujeres, probablemente feministas, marchando con pancartas y símbolos de género femenino.
2. Label: YES

Este meme parece utilizar una escena de "El Señor de los Anillos" para retratar de manera negativa o alarmista la llegada o acción de un grupo de mujeres manifestantes, sugiriendo que son una amenaza o algo que debe ser observado con precaución, lo cual puede interpretarse como un refuerzo de estereot

In [ ]:
spanish_memes_df_test.set_index('id_EXIST', inplace=True)

file_path = './data/Spanish_Meme_GPT_Description_test.json'

spanish_memes_df_test.to_json(file_path, orient='index', indent=4)

print(f'Data saved to {file_path}')

Data saved to Spanish_Meme_GPT_Description_test.json


In [ ]:
import json

spanish_answers_test = {key: {"answer": value} for key, value in spanish_answers_test.items()}

with open('./data/Spanish_Meme_GPT_Initial_Answers_test.json', 'w') as json_file:
    json.dump(spanish_answers_test, json_file, indent=4)

In [ ]:
# process English meme and Spanish meme seperately
spanish_path = "./data/Spanish_memes/"
spanish_memes_df, spanish_answers = create_df(spanish_path)

In [ ]:
spanish_memes_df.set_index('id_EXIST', inplace=True)


file_path = './data/Spanish_Meme_GPT_Description.json'


spanish_memes_df.to_json(file_path, orient='index', indent=4)

print(f'Data saved to {file_path}')

In [ ]:
import json

spanish_answers = {key: {"answer": value} for key, value in spanish_answers.items()}

with open('./data/Spanish_Meme_GPT_Initial_Answers.json', 'w') as json_file:
    json.dump(spanish_answers, json_file)

Process the test data and save it in json file

In [ ]:
# create two directories to store the memes
os.makedirs('./data/Spanish_memes_test', exist_ok=True)
os.makedirs('./data/English_memes_test', exist_ok=True)

memes_path = './data/EXIST 2024 Memes Dataset/test/memes'
for filename in os.listdir(memes_path):

    if filename.startswith('3'):

        shutil.copy(os.path.join(memes_path, filename), os.path.join('./data/Spanish_memes_test', filename))
    elif filename.startswith('4'):

        shutil.copy(os.path.join(memes_path, filename), os.path.join('./data/English_memes_test', filename))

In [ ]:
english_test_path = "./data/English_memes_test"
english_test_memes_df, english_test_answers = create_df(english_test_path)

In [ ]:
english_test_memes_df.set_index('id_EXIST', inplace=True)

file_path = './data/English_Meme_GPT_Description_test.json'

english_test_memes_df.to_json(file_path, orient='index', indent=4)

print(f'Data saved to {file_path}')

In [ ]:
import json
english_test_answers = {key: {"answer": value} for key, value in english_test_answers.items()}
with open('./data/English_Meme_GPT_Initial_Answers_test.json', 'w') as json_file:
    json.dump(english_test_answers, json_file)

In [ ]:
# process English meme and Spanish meme seperately
spanish_test_path = "./data/English_memes_test"
spanish_test_memes_df, spanish_test_answers = create_df(spanish_test_path)

In [ ]:
spanish_test_memes_df.set_index('id_EXIST', inplace=True)
file_path = './data/Spanish_Meme_GPT_Description_test.json'

spanish_test_memes_df.to_json(file_path, orient='index', indent=4)

print(f'Data saved to {file_path}')

In [ ]:
import json

spanish_test_answers = {key: {"answer": value} for key, value in spanish_answers.items()}

with open('./data/Spanish_Meme_GPT_Initial_Answers_test.json', 'w') as json_file:
    json.dump(spanish_test_answers, json_file)